In [ ]:
cd 'C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\Biofilm growth code\GSWL code\Biochemical-databases'

# TECRDB scraping 

In [ ]:
# import libraries
from bs4 import BeautifulSoup
from time import sleep
import requests
import datetime
import pandas
import numpy
import glob
import math
import os
import re

# defining the website
root_url = "https://randr.nist.gov/enzyme/DataDetails.aspx?ID="
end_url = "&finalterm=&data=enzyme"
#===========================================================================================================================

# identify the table and rows of pertinent data
bs = BeautifulSoup(open('Enzyme Thermodynamic Database.html'), 'lxml')
table = bs.find("table", attrs = {'id': 'MainBody_gvSearch'})
body = table.find_all("tr")

# defining the boundaries of the dataframe section
index_range = 12000
index_count = 0
loop_count = 0 
output_loop = 1
total_entries = math.floor(1*len(body))

# loop through the enzyme id values 
scraped_csv_folder_path = './individual scraping/'
name_iteration = 0
enzyme_iteration = 0
for row in range(1, total_entries):   
    
    # parsing the reaction names and strings   
    enzyme_name = body[row].find('span', attrs = {'id': 'MainBody_gvSearch_lblEnzyme_%s' %(name_iteration)}).text
    reaction = body[row].find('span', attrs = {'id': 'MainBody_gvSearch_lblReaction_%s' %(name_iteration)}).text
    id_value = body[row].find("a").text
    name_iteration += 1
    
    # defining the soup
    total_url = root_url + id_value + end_url
    soup = requests.get(total_url).text
    bs = BeautifulSoup(soup, 'lxml')
    
    # scrape the table and header information
    tables1 = bs.find_all("table", attrs = {"id": "MainBody_extraData"})
    print(id_value, '\t: ', loop_count, '\t, ', len(tables1))
    if len(tables1) != 1:
        continue

    body1 = tables1[0].find_all("tr")
    body_rows1 = body1[1:]
    heads = body1[0]

    headings = ['Enzyme:', 'Reaction:']
    for head in heads.find_all("th"):
        head = (head.text).rstrip("\n")
        headings.append(head)

    total_rows = []
    for row_number in range(len(body_rows1)):
        each_row = [enzyme_name, reaction]
        for row_element in body_rows1[row_number].find_all("td"):
            row_refined = re.sub("(\xa0)|(\n)|,","",row_element.text)
            each_row.append(row_refined)
        total_rows.append(each_row)
    
    # create a dataframe
    index_list_body = range(index_count+lower_bound-1, len(body_rows1)+index_count+lower_bound-1)
    bs_dataframe_table1 = pandas.DataFrame(data = total_rows, columns = headings, index = index_list_body)
    bs_dataframe_table1.drop(bs_dataframe_table1.columns[len(bs_dataframe_table1.columns)-1], axis=1, inplace=True)
    '''display(bs_dataframe_table1)'''
    
#===========================================================================================================================

    # scrape additional information 
    tables = bs.find_all("table", attrs={"id": "MainBody_DataList1"})
    if len(tables) == 0:
        print('Failed reference ID: ', id_value)
        continue
        
    body2 = tables[0].find_all("tr")
    body_rows2 = body2[1:]

    each_row2 = []
    for row in range(len(body_rows2)):
        for row_element in body_rows2[row].find_all("td"):
            row_refined2 = re.sub("(\xa0)|(\n)|,","",row_element.text)
            each_row2.append(row_refined2)

    information_entries_list = []
    information_values_list = []
    column_count = 0
    for i, element in enumerate(each_row2):
        if i == 0 or i % 2 == 0:
            information_entries_list.append(element)
            column_count += 1
        else:
            information_values_list.append(element)
            column_count += 1
    column_count /= 2

    # create the dataframe and refine the columns
    index_list_reference = range(index_count+lower_bound-1, index_count+1+lower_bound-1)
    bs_dataframe_pretable2 = pandas.DataFrame(data = [information_values_list], columns = information_entries_list, index = index_list_reference)
    bs_dataframe_pretable2.drop(bs_dataframe_pretable2.columns[len(bs_dataframe_pretable2.columns)-2], axis=1, inplace=True)
    bs_dataframe_pretable2.drop(bs_dataframe_pretable2.columns[len(bs_dataframe_pretable2.columns)-1], axis=1, inplace=True)
    
#===========================================================================================================================

    # merge the dataframes of this loop
    this_dataframe = bs_dataframe_table1.join(bs_dataframe_pretable2)
    this_dataframe.index.name = 'index'
    
    # iteratively coalesce the new dataframe into the old dataframe 
    if loop_count == 0:
        old_dataframe = this_dataframe
        old_dataframe.index.name = 'index'
        
    elif loop_count > 0:
        these_columns = []
        for column in this_dataframe:
            these_columns.append(column)
            
        old_columns = []
        for existing_column in old_dataframe:
            old_columns.append(existing_column)
            
        common_columns = list(set(these_columns).intersection(old_columns))
        
        # amalgamate the dataframe with the pre-existing dataframe
        current_dataframe = old_dataframe.merge(this_dataframe, on = common_columns, how = 'outer')
        old_dataframe = current_dataframe  
        
    # amalgamate the dataframe with the pre-existing dataframe
    index_count += len(body_rows1)
    
    time_delay = 0
    sleep(time_delay)
    
    max_referenes_per_csv = 0
    if loop_count == max_referenes_per_csv:
        id_value = re.sub('(/)', '-', id_value)
        output = '{}/{}, {}.csv'.format(scraped_csv_folder_path, datetime.date.today(), id_value, output_loop)
        while os.path.exists(output):
            output_loop += 1
            output = '{}/2021-05-06_{}.csv'.format(scraped_csv_folder_path, datetime.date.today(), id_value, output_loop)
            
        old_dataframe.to_csv(output)
        loop_count = 0 
    else:        
        loop_count += 1 


# create the total list of dataframes
files = glob.glob(os.path.join(scraped_csv_folder_path, '*.csv'))
total_dataframes = []
for file in files:
    dfn = pandas.read_csv(file)
    total_dataframes.append(dfn)
    
# combine the total set of dataframes
combined_df = pandas.DataFrame()
combined_df = pandas.concat(total_dataframes)
display(combined_df)

# refine the dataframe 
combined_df = combined_df.fillna(' ')

middle_dataframe_columns = ['T(K)', 'pH ', 'K<sub>c</sub>\' ', 'δ<sub>r</sub>H\'<sup>o</sup>(kJ.mol<sup>-1</sup>)', 'Km\'']
left_dataframe_columns = ['index', 'Enzyme:', 'Reaction:', 'Reference:', 'Reference ID:'] 
right_dataframe_columns = list(set(combined_df.columns) - set(left_dataframe_columns) - set(middle_dataframe_columns))
defined_columns = left_dataframe_columns + middle_dataframe_columns + right_dataframe_columns  
final_dataframe = combined_df.reindex(columns = defined_columns)

# export the final dataframe of values
'''final_dataframe.to_csv('{}_concatenated scraped NIST enzymes_01.csv'.format(datetime.date.today()))'''

# Data assemblage and organization

In [2]:
#import libraries
from datetime import date 
from time import sleep
import pandas
import numpy
import math
import json
import re

empty_cell = ['nan', 'NaN', 'none', 'not given', 'unknown', '0', '', None, ' ']

# dataframes definition
combined_dataframe = pandas.read_csv("2021-05-06_concatenated scraped NIST enzymes_01.csv")
df = combined_dataframe.astype(str)


# combine columns of the complete scraped excel file 
combined_columns = []
regex_column_searches = ['(?i)(^K)', '(?i)(^K)', '(ë«|Î\´|δ)', '(I<sub>c)']
base_columns = ['K<sub>c</sub>\' ', 'K<sub>c</sub>\' ', 'δ<sub>r</sub>H(cal)/kJ mol<sup>-1</sup>)', 'I<sub>c</sub>(mol dm<sup>-3</sup>)']
print('\nColumns:\n', '='*len('Columns:'))
for this_column in df:
    print(this_column)
    for index, row in df.iterrows():
        for search in range(len(regex_column_searches)):
            
            # combine similar columns
            if re.search(regex_column_searches[search], this_column) and not re.search('(Km\')', this_column):
                if str(df.at[index, this_column]) not in empty_cell: 
                    if str(df.at[index, base_columns[search]]) in empty_cell:
                        df.at[index, base_columns[search]] = str(df.at[index, this_column])

                    if str(df.at[index, this_column]) not in empty_cell:
                        if str(df.at[index, base_columns[search]]) != str(df.at[index, this_column]):
                            df.at[index, base_columns[search]] = str(df.at[index, base_columns[search]]) + ' & ' + str(df.at[index, this_column])

                if this_column !=  base_columns[search]:
                    if this_column not in combined_columns:
                        combined_columns.append(this_column)

        if re.search('(?<=c\()(\w+\d?\+?)(?<!,)', this_column):
            solute = str(re.search('(?<=c\()(\w+\d?\+?)(?<!,)', this_column).group(1))
                
            if str(df.at[index, this_column]) not in empty_cell:
                if str(df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)']) in empty_cell:
                    df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)'] = str(df.at[index, this_column]) + ' ' + solute

                if str(df.at[index, this_column]) not in empty_cell:
                    if str(df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)']) != (str(df.at[index, this_column]) or str(df.at[index, this_column]) + ' ' + solute):
                        df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)'] = str(df.at[index, 'c(glycerol,mol dm<sup>-3</sup>)']) + ' & ' + str(df.at[index, this_column]) + ' ' + solute

            if this_column !=  'c(glycerol,mol dm<sup>-3</sup>)':
                if this_column not in combined_columns:
                    combined_columns.append(this_column)

        if re.search('(?<=m\()(\w+\d?\+?)(?<!,)', this_column):
            solute = str(re.search('(?<=m\()(\w+\d?\+?)(?<!,)', this_column).group(1))
                
            if str(df.at[index, this_column]) not in empty_cell:
                if str(df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)']) in empty_cell:
                    df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)'] = str(df.at[index, this_column]) + ' ' + solute
                if str(df.at[index, this_column]) not in empty_cell:
                    if str(df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)']) != (str(df.at[index, this_column]) or str(df.at[index, this_column]) + ' ' + solute):
                        df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)'] = str(df.at[index, 'm(MgCl2,mol.kg<sup>-1</sup>)']) + ' & ' + str(df.at[index, this_column]) + ' ' + solute

            if this_column !=  'm(MgCl2,mol.kg<sup>-1</sup>)':
                if this_column not in combined_columns:
                    combined_columns.append(this_column)

        buffer_columns = ['buffer(mol dm<sup>-3</sup>)', 'buffer and/or salt ', 'media ', 'buffer ']
        if this_column in buffer_columns:
            if str(df.at[index, this_column]) not in empty_cell:
                if str(df.at[index, 'Buffer:']) in empty_cell:
                    df.at[index, 'Buffer:'] = str(df.at[index, this_column])

                if str(df.at[index, 'Buffer:']) not in empty_cell:
                    if not re.search(re.escape(str(df.at[index, this_column])), str(df.at[index, 'Buffer:'])):
                        df.at[index, 'Buffer:'] = str(df.at[index, 'Buffer:']) + '  +  ' + str(df.at[index, this_column])

            if this_column !=  'Buffer:':
                if this_column not in combined_columns:
                    combined_columns.append(this_column)

        solution_columns = ['salt ', 'cosolvent ', 'added solute ', 'protein ', 'added solute ', 'percent(dimethyl sulfoxide) ', 'p(MPa)', 'pMg ']
        if this_column in solution_columns:
            if str(df.at[index, this_column]) not in empty_cell:
                if str(df.at[index, 'solvent ']) in empty_cell:
                    if str(this_column) == 'p(MPa)':
                        df.at[index, 'solvent '] = str(df.at[index, this_column]) + ' megapascals'  
                    elif str(this_column) == 'pMg ':
                        df.at[index, 'solvent '] = str(df.at[index, this_column]) + ' = -log[Mg+2]'   
                    elif str(this_column) == 'percent(dimethyl sulfoxide) ':
                        df.at[index, 'solvent '] = str(df.at[index, this_column]) + ' % DMSO'   
                    else:
                        df.at[index, 'solvent '] = str(df.at[index, this_column])

                if str(df.at[index, 'solvent ']) not in empty_cell:
                    if not re.search(re.escape(str(df.at[index, this_column])), str(df.at[index, 'solvent '])):
                        df.at[index, 'solvent '] = str(df.at[index, 'solvent ']) + '  +  ' + str(df.at[index, this_column])

            if this_column !=  'solvent ':
                if this_column not in combined_columns:
                    combined_columns.append(this_column)
            
# rename the base columns
df.rename(columns = {'c(glycerol,mol dm<sup>-3</sup>)':'solutes [mol / dm^3]', 
                     'I<sub>c</sub>(mol dm<sup>-3</sup>)':'Ionic strength [mol / dm^3]', 
                     'T(K)':'T [K]', 
                     'I<sub>m</sub>(mol.kg<sup>-1</sup>)':'Ionic strength [mol / kg]', 
                     'm(MgCl2,mol.kg<sup>-1</sup>)':'solutes [mol / kg]', 
                     'solvent ':'Experimental conditions', 
                     'K<sub>c</sub>\' ':'Keq', 
                     'δ<sub>r</sub>H(cal)/kJ mol<sup>-1</sup>)':'Enthalpy [kJ / mol]',
                     'Km\' ':'Km'},
          inplace = True)
          
    
# delete the combined columns
print('\nDeleted columns:')
for column in combined_columns:
    print(column)
    del df[column]
del df['Km\'']
    
    
# export the dataframe to csv 
'''df.to_csv("{}_vetted & reorganized NIST database_01.csv".format(date.today()))'''


# acquire a list of all enzymes
enzyme_list = []
empty_cell = ['nan', 'NaN', 'none', 'not given', '', ' ', None, numpy.nan]
for index, row in df.iterrows():
    if df.at[index, 'Enzyme:'] not in enzyme_list and df.at[index, 'Enzyme:'] not in empty_cell:
        enzyme_list.append(df.at[index, 'Enzyme:'])      
        
enzymes = []
for original_enzyme in enzyme_list:
    enzyme_name = re.search('(\w.*)',original_enzyme)
    enzymes.append(enzyme_name.group())
    
# count down for processing and organizing the data
output_json_name = '{}_TECR_consolidated.json'.format(date.today())

count = 0
'''while not os.path.exists(output_json_name):
    if count % 2:
        mark = '.'
    else:
        mark = ''
    print('The data is being assembled and organized ..{}'.format(mark), end = '\r')
    count += 1
    sleep(1)'''
    
'''necessary_time_to_organize = 460
for time in reversed(range(necessary_time_to_organize)):
    print('The database data is being assembled and organized ... {}'.format(time), end = '\r')
    time -= 1
    sleep(1)'''
    
# the data is processed and organized from the originally scraped data 
data_per_enzyme = {}
total_entries = 3979
count = 1
        
for enzyme in enzymes:
    
    # print the progress statement
    print('The data is being assembled and organized ... {}/425'.format(count), end = '\r')
    
    # lists of the database variables
    keq_values_per_enzyme = []
    km_values_per_enzyme = []
    enthalpy_values_per_enzyme = []
    temperatures_per_enzyme = []
    phs_per_enzyme = []
    
    # lists of identifying whether the reference contains the identified variable 
    references_of_an_enzyme = []
    reaction_of_an_enzyme = []
    km_in_the_reference = []
    enthalpy_in_the_reference = []
    keqs_in_a_reference = []
    for index, row in df.iterrows():
        iteration = 0
        if df.at[index, 'Enzyme:'] == ' %s' %(enzyme):
            reaction_of_an_enzyme.append(df.at[index, 'Reaction:'])
            if df.at[index, 'Reference:'] not in empty_cell:
                references_of_an_enzyme.append(df.at[index, 'Reference:'])
            elif df.at[index, 'Reference:'] in empty_cell:
                references_of_an_enzyme.append('Ibid')
            
            # clean keqs are added to a list
            if df.at[index, 'Keq'] not in empty_cell:
                cleaned_keq = re.search('(\-?\d+\.?\d*)', '%s' %(df.at[index, 'Keq']))
                keq_values_per_enzyme.append(float(cleaned_keq.group())) 
                keqs_in_a_reference.append('True')
                temperatures_per_enzyme.append(df.at[index, 'T [K]'])
                if df.at[index, 'pH '] not in empty_cell:
                    phs_per_enzyme.append(df.at[index, 'pH '])
                elif df.at[index, 'pH '] in empty_cell:
                    phs_per_enzyme.append('nan')
            
            elif df.at[index, 'Keq'] in empty_cell:
                keqs_in_a_reference.append('False')    
                
            # clean kms are added to a list
            if df.at[index, 'Km'] not in empty_cell:
                cleaned_km = re.search('(\-?\d+\.?\d*)', '%s' %(df.at[index, 'Km']))
                km_values_per_enzyme.append(float(cleaned_km.group())) 
                if df.at[index, 'Keq'] in empty_cell:
                    temperatures_per_enzyme.append(df.at[index, 'T [K]'])
                    if df.at[index, 'pH '] not in empty_cell:
                        phs_per_enzyme.append(df.at[index, 'pH '])
                    elif df.at[index, 'pH '] in empty_cell:
                        phs_per_enzyme.append('nan')
                    
            elif df.at[index, 'Km'] in empty_cell:
                km_in_the_reference.append('False')
                
            # clean enthalpy values are added to a list
            if df.at[index, 'Enthalpy [kJ / mol]'] not in empty_cell:
                cleaned_enthalpy = re.search('(\-?\d+\.?\d*)', '%s' %(df.at[index, 'Enthalpy [kJ / mol]']))
                enthalpy_in_the_reference.append('True')
                enthalpy_values_per_enzyme.append(float(cleaned_enthalpy.group())) 
                if df.at[index, 'Keq'] in empty_cell and df.at[index, 'Km'] in empty_cell:
                    temperatures_per_enzyme.append(df.at[index, 'T [K]'])
                    if df.at[index, 'pH '] not in empty_cell:
                        phs_per_enzyme.append(df.at[index, 'pH '])
                    elif df.at[index, 'pH '] in empty_cell:
                        phs_per_enzyme.append('nan')
                    
            elif df.at[index, 'Enthalpy [kJ / mol]'] in empty_cell:
                enthalpy_in_the_reference.append('False')
                
                
            #loop through the unlabeled rows of each enzyme
            while df.at[index + iteration, 'Enzyme:'] in empty_cell:
                if df.at[index, 'Keq'] not in empty_cell:
                    cleaned_keq = re.search('(\-?\d+\.?\d*)', '%s' %(df.at[index, 'Keq']))
                    keq_values_per_enzyme.append(float(cleaned_keq.group())) 
                    keqs_in_a_reference.append('True')
                    temperatures_per_enzyme.append(df.at[index, 'T [K]'])
                    if df.at[index, 'pH '] not in empty_cell:
                        phs_per_enzyme.append(df.at[index, 'pH '])
                    elif df.at[index, 'pH '] in empty_cell:
                        phs_per_enzyme.append('nan')

                elif df.at[index, 'Keq'] in empty_cell:
                    keqs_in_a_reference.append('False')  
                    
                #clean kms are added to a list
                if df.at[index, 'Km'] not in empty_cell:
                    cleaned_km = re.search('(\-?\d+\.?\d*)', '%s' %(df.at[index, 'Km']))
                    km_in_a_reference.append('True')
                    km_values_per_enzyme.append(float(cleaned_km.group())) 
                    if df.at[index, 'Keq'] in empty_cell:
                        temperatures_per_enzyme.append(df.at[index, 'T [K]'])
                        if df.at[index, 'pH '] not in empty_cell:
                            phs_per_enzyme.append(df.at[index, 'pH '])
                        elif df.at[index, 'pH '] in empty_cell:
                            phs_per_enzyme.append('nan')

                elif df.at[index, 'Km'] in empty_cell:
                    km_in_the_reference.append('False')
                    
                # clean enthalpy values are added to a list
                if df.at[index, 'Enthalpy [kJ / mol]'] not in empty_cell:
                    cleaned_ethalpy = re.search('(\-?\d+\.?\d*)', '%s' %(df.at[index, 'Enthalpy [kJ / mol]']))
                    enthalpy_in_the_reference.append('True')
                    enthalpy_values_per_enzyme.append(float(cleaned_enthalpy.group())) 
                    if df.at[index, 'Keq'] in empty_cell and df.at[index, 'Km'] in empty_cell:
                        temperatures_per_enzyme.append(df.at[index, 'T [K]'])
                        if df.at[index, 'pH '] not in empty_cell:
                            phs_per_enzyme.append(df.at[index, 'pH '])
                        elif df.at[index, 'pH '] in empty_cell:
                            phs_per_enzyme.append('nan')

                elif df.at[index, 'Enthalpy [kJ / mol]'] in empty_cell:
                    enthalpy_in_the_reference.append('False')
                    
                #proceed to the next loop
                if iteration + index < total_entries:
                    iteration += 1

    #processing the average and standard deviation Keq values
    if len(keq_values_per_enzyme) != 0:
        average_keq_per_enzyme = sum(keq_values_per_enzyme) / len(keq_values_per_enzyme)
        standard_deviation_keq_per_enzyme = math.sqrt(sum([(x - average_keq_per_enzyme)**2 for x in keq_values_per_enzyme]) / len(keq_values_per_enzyme))
    elif len(keq_values_per_enzyme) == 0:
        average_keq_per_enzyme = 'nan'
        standard_deviation_keq_per_enzyme = 'nan'
        
    #processing the average and standard deviation Km values
    if len(km_values_per_enzyme) != 0:
        average_km_per_enzyme = sum(km_values_per_enzyme) / len(km_values_per_enzyme)
        standard_deviation_km_per_enzyme = math.sqrt(sum([(x - average_km_per_enzyme)**2 for x in km_values_per_enzyme]) / len(km_values_per_enzyme))
    elif len(km_values_per_enzyme) == 0:
        average_km_per_enzyme = 'nan'
        standard_deviation_km_per_enzyme = 'nan' 
        
    #processing the average and standard deviation enthalpy values
    if len(enthalpy_values_per_enzyme) != 0:
        average_enthalpy_per_enzyme = sum(enthalpy_values_per_enzyme) / len(enthalpy_values_per_enzyme)
        standard_deviation_enthalpy_per_enzyme = math.sqrt(sum([(x - average_enthalpy_per_enzyme)**2 for x in enthalpy_values_per_enzyme]) / len(enthalpy_values_per_enzyme))
    elif len(enthalpy_values_per_enzyme) == 0:
        average_enthalpy_per_enzyme = 'nan'
        standard_deviation_enthalpy_per_enzyme = 'nan' 
        
    #store the information into a nested dictionary structure
    data_per_enzyme[enzyme] = {'reaction':reaction_of_an_enzyme,
                               'experimental temperatures':temperatures_per_enzyme,
                               'experimental phs':phs_per_enzyme,
                               'keq reference':references_of_an_enzyme,
                               'Keq':{'keq values in the reference':keqs_in_a_reference,
                                       'keqs':keq_values_per_enzyme, 
                                       'keq quantity':len(keq_values_per_enzyme), 
                                       'keq average':average_keq_per_enzyme, 
                                       'keq standard deviation':standard_deviation_keq_per_enzyme},
                               'Km':{'km values in the reference':km_in_the_reference,
                                    'km values':km_values_per_enzyme,
                                    'km average':average_km_per_enzyme,
                                    'km standard deviation':standard_deviation_km_per_enzyme},
                               'Enthalpy':{'enthalpy values in the reference':enthalpy_in_the_reference,
                                         'enthalpy values':enthalpy_values_per_enzyme,
                                         'enthalpy average':average_enthalpy_per_enzyme,
                                         'enthalpy standard deviation':standard_deviation_enthalpy_per_enzyme}
                              }
    count += 1

#export the database dictionary as a JSON file
with open(output_json_name, 'w') as output:
    json.dump(data_per_enzyme, output, indent = 4)


Columns:
Unnamed: 0
index
Enzyme:
Reaction:
Reference:
Reference ID:
T(K)
pH 
K<sub>c</sub>' 
δ<sub>r</sub>H'<sup>o</sup>(kJ.mol<sup>-1</sup>)
Km'
δ<sub>r</sub>H(kJ.mol<sup>-1</sup>)
K<sub>c</sub> 
m(KCl,mol.kg<sup>-3</sup>)
added solute 
δ<sub>r</sub>H<sup>o</sup>(kJ.mol<sup>-1</sup>)
cosolvent 
δ<sub>r</sub>H?(kJ.mol<sup>-1</sup>)
Km' 
media 
Method:
salt 
c(Mg)tot(mol dm<sup>-3</sup>)
c(Tris,mol dm<sup>-3</sup>)
p(MPa)
c(NaCl,mol dm<sup>-3</sup>)
I<sub>c</sub> 
c(MgCl2,mol dm<sup>-3</sup>)
buffer and/or salt 
Buffer:
K 
c(CaCl2,mol dm<sup>-3</sup>)
buffer(mol dm<sup>-3</sup>)
solvent 
c(MgSO4,mol dm<sup>-3</sup>)
I<sub>c</sub>(kJ.mol<sup>-1</sup>)
δ<sub>r</sub>H(cal)/(n<sup>-1</sup>
kJ.mol<sup>-1</sup>)
c(Mg2+,mol dm<sup>-3</sup>)
c(salt,mol dm<sup>-3</sup>)
pMg 
δ<sub>r</sub>H(cal)/,kJ.mol<sup>-1</sup>)
EC Value:
δ<sub>r</sub>Hcal,kJ.mol<sup>-1</sup>)
K' 
c(phosphate,mol dm<sup>-3</sup>)
percent(dimethyl sulfoxide) 
c(total substrate,mol dm<sup>-3</sup>)
δ<sub>r</sub>H(cal) 
m(MgC

# ModelSEED reconfiguration

In [3]:
# import the requisite libraries
from ftfy import fix_encoding
from datetime import date
import pandas
from numpy import nan
import json
import re
import io


# import and define reused information
tecrdb_compounds = json.load(open('2021-05-09_APF_nist thermodynamically described compounds for ModelSEED_01.json'))
tecrdb_reactions = json.load(open('2021-05-09_APF_nist thermodynamically described reactions for ModelSEED_01.json'))
nist = json.load(open('2021-05-06_NIST consolidated.json'))
empty_cells = ['null', ' null', nan, 'None', None, ""]

# create a condensed dictionary of all ModelSEED compound names and aliases
modelseed = json.load(open('compounds.json'))
modelseed_compounds = {}
all_names = []
for compound2 in modelseed:
    if compound2['is_obsolete'] == 0:
        try:
            for entry in compound2['aliases']:
                if re.search('Name:', entry):
                    alternative_names = re.sub('Name: ', '', entry).split('; ')
                    alternative_names.append(compound2['abbreviation'])
                    main_name = alternative_names[0].lower()
                if re.search('KEGG:', entry):
                    kegg_id = re.sub('KEGG: ', '', entry).split('; ')

        except:
            main_name = compound2['name'].lower()
            alternative_names = compound2['abbreviation']
            
        modelseed_compounds[main_name] = {'ModelSEED ID': compound2['id'],
                                          'KEGG ID': kegg_id,
                                          'alternative names': alternative_names}

# export the dictionary of modelseed compounds with the respective information
'''with open('{}_modelseed compounds & alternative names_01.json'.format(date.today()), 'w') as output:
    json.dump(modelseed_compounds, output, indent = 4)'''
    
    
# parse the modelseed database list of compounds
all_names = []
for compound in modelseed_compounds:
    if type(modelseed_compounds[compound]['alternative names']) is list:
        all_names.append([compound.lower()] + modelseed_compounds[compound]['alternative names'])
    elif type(modelseed_compounds[compound]['alternative names']) is str:
        all_names.append([compound.lower()] + list(modelseed_compounds[compound]['alternative names']))
    else:
        print('ERROR: The database entry for {} is an unpredicted structure.')
        
total_modelseed_names = []
for name_list in all_names:
    total_modelseed_names += name_list 
total_modelseed_names_string = ',\t'.join(list(set(total_modelseed_names)))   


# export the string of identified ModelSEED compound names
'''with open('2021-06-27_total_modelseed_names_string_01.txt', 'w') as output:
    output.write(modelseed_abbreviations_string)
    output.close()'''
modelseed_abbreviations = modelseed_compounds.keys()
modelseed_abbreviations_string = ',\t'.join(modelseed_abbreviations)
'''with open('2021-06-27_modelseed abbreviation string_01.txt', 'w') as output:
    output.write(modelseed_abbreviations_string)
    output.close()'''


# determine the set of described and undescribed compounds of the TECRDB in the ModelSEED database 
described_compounds = {}
undescribed_compounds = []
unknown_compounds = []
repeated_compounds = set()
for compound in tecrdb_compounds:    
    compound = re.sub('(\(\+\))', '\(\+\)', compound)
    compound = compound.lower()
    if compound not in (described_compounds.keys() or undescribed_compounds or unknown_compounds):
        if re.search('((^|,\t){},\t)'.format(compound), total_modelseed_names_string, re.IGNORECASE):
            if re.search('((^|,\t){},\t)'.format(compound), modelseed_abbreviations_string, re.IGNORECASE):
                compound = fix_encoding(compound)
                print('identified abbreviation: ', compound)
                #re.sub(',\s(\w)\w+', '\U\1', ', {}, '.format(compound))
                compound = re.sub('(\\\)', '', compound)
                described_compounds[compound] = modelseed_compounds[compound]
            else:
                for name_list in all_names:
                    name_list_string = ',\t'.join(name_list)
                    if re.search('((^|,\t){},\t)'.format(compound), name_list_string, re.IGNORECASE):
                        root_compound = name_list[0]
                        root_compound = fix_encoding(root_compound)
                        print('identified alias: ', compound)
                        #print(name_string)
                        described_compounds[compound] = modelseed_compounds[root_compound]
    
                if compound not in (described_compounds.keys() or unknown_compounds):
                    unknown_compounds.append(compound)
                    print('ERROR 1: {} is structurally predicted.'.format(compound))
                    
        else:
            if compound not in undescribed_compounds:
                compound = fix_encoding(compound)
                undescribed_compounds.append(compound)
                print('--> undescribed compound in ModelSEED, {} '.format(compound))
            else:
                if compound not in unknown_compounds:
                    unknown_compounds.append(compound)
                    print('ERROR 2: {} is structurally predicted.'.format(compound))
                    
    else:
        repeated_compounds.add(compound)
                
            
# view the parsed data
print('\n\n')
compounds_list = []
for compound in tecrdb_compounds.keys():
    compound = fix_encoding(compound)
    compounds_list.append(compound.lower())                          

undescribed_compounds_set = set()
duplicated_compounds = []
for compound in undescribed_compounds:
    compound = fix_encoding(compound)
    compound = compound.strip()
    undescribed_compounds_set.add(compound)

print('described compounds: ', len(described_compounds.keys()))
print('undescribed_compounds: ', len(undescribed_compounds))
print('unknown compounds: ', len(unknown_compounds))
print('repeated compounds: ', len(repeated_compounds))
print('total compounds: ', len(set(compounds_list)))

# view any missing compounds
modeled_compounds = undescribed_compounds_set.union(set(described_compounds.keys())) 
modeled_compounds_set = set()
for compound in modeled_compounds:
    compound = fix_encoding(compound)
    compound = compound.strip()
    modeled_compounds_set.add(compound)
    
print('\n')
print('modeled compounds: ', len(modeled_compounds_set))
missing_compounds = set(compounds_list) - modeled_compounds_set
print('missing compounds: ', len(set(missing_compounds)))
for compound in missing_compounds:
    print(compound)
print('\n\n')
    
# export the acquired sets of described and undescribed compounds
'''with open('{}_APF_described compounds in ModelSEED_01.json'.format(date.today()), 'w') as output:
    json.dump(described_compounds, output, indent = 5)'''

'''with open('{}_APF_undescribed compounds in ModelSEED_01.csv'.format(date.today()), 'w', encoding="utf-8") as output:
    undescribed_dataframe = pandas.DataFrame(undescribed_compounds, columns = ['undescribed compounds'])
    undescribed_dataframe.fillna('', inplace = True)
    undescribed_dataframe.to_csv(output)'''

'''with io.open('2021-05-09_APF_NIST compounds absent from ModelSEED_01.csv', 'w', encoding="utf-8") as output:
    undescribed_compund_csv = pandas.DataFrame(undescribed_compounds)
    undescribed_compund_csv.to_csv(output)'''

# define functions that parse the data later
def isfloat(entry):
    try:
        value = float(entry)
        return True
    except ValueError:
        value = ''
        return False
    
def average(list_entry):
    if len(list_entry) > 0:
        average =  sum(list_entry) / len(list_entry)
    else:
        average = 'nan'
    return average

# define the complete set of enzymes that are described by the database
enzymes = set()
for enzyme, information in nist.items():
    enzymes.add(enzyme.lower())
    
# arrange the data for the described compounds
described_enzyme_data = {}
for enzyme, information in nist.items():
    if enzyme.lower() in enzymes:
        
        # determine the set of unique reactions for each enzyme
        unique_reactions = set()
        for reaction in information['reaction']:
            unique_reactions.add(reaction)
        
        # calculate the average temperature 
        temperatures = []
        for temperature in information['experimental temperatures']:
            temperature = re.sub('(~)', '', temperature)
            if isfloat(temperature):
                temperatures.append(float(temperature))
        average_temperature = average(temperatures)
            
        # calculate the average ph 
        phs = []
        for ph in information['experimental phs']:
            ph = re.sub('(~|nan)', '', ph)
            if isfloat(ph):
                phs.append(float(ph))
        average_ph = average(phs)
                
    described_enzyme_data[enzyme] = {'unique reactions': unique_reactions,
                                    'average keq': information['Keq']['keq average'],
                                    'average enthalpy': information['Enthalpy']['enthalpy average'],
                                    'average temperature kelvin': average_temperature,
                                    'average pH': average_ph}

    
# export the dictionary of unique reactions and data for each enzyme
'''with open('{}_APF_ModelSEED described enzymes_01.json'.format(date.today()), 'w') as output:
    json.dump(described_enzyme_data, output, indent = 4)'''


# the reaction strings are completely substituted with ModelSEED compound IDs
described_compounds_list = []
for compound in described_compounds:
    described_compounds_list.extend([compound.lower()] + list(described_compounds[compound]['alternative names']))
described_compounds_string = ',\t'.join(list(set(described_compounds_list))) 
'''with open('described compounds matrix.txt', 'w') as output:
    output.write('\n'.join(described_compounds_list))
    output.close()'''
    
total_complete_reactions = []
exchanged_reactions = set()
to_modelseed_reactions = {}
reactions = {}

reaction_id_pairs = {}
id_number = 0
repeated_reactions = set()
for enzyme, information in described_enzyme_data.items():
    complete_reactions = []
    described_reactions = set()
    undescribed_reactions = set()
    print('\n', enzyme)
    for reaction in information['unique reactions']:
        reaction = fix_encoding(reaction)
        reaction = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', reaction)

        # parse the reactants and products for the specified reaction string
        reactants_list = []
        products_list = []
        compounds_list = []
        if re.search('(=)', reaction):
            reaction_split = reaction.split('=')
            reactants_list = reaction_split[0].split(' + ')
            products_list = reaction_split[1].split(' + ')
            reactants = []
            stoich_coefficient = re.compile('(\d\s|\d\/\d\s)')
            for element in reactants_list:
                element = element.strip()
                if re.search('(\d\s|\d\/\d\s)', element):
                    coefficient = stoich_coefficient.findall(element)[0]
                    if re.search('\/', coefficient):
                        numerator = re.findall('\d', coefficient)[0]
                        denominator = re.findall('\d', coefficient)[1]
                        numerical_coefficient = float(numerator) / float(denominator)
                        coefficient = '({}) '.format(numerical_coefficient)
                    else:
                        coefficient = '({}) '.format(re.findall('\d', coefficient)[0])
                else:
                    coefficient = ''
                    
                element = re.sub('(\d\s|\d\/\d\s)', '', element)
                if element in described_compounds_list:
                    for compound, aliases in described_compounds.items():
                        if element.lower() == compound.lower() or element in aliases['alternative names']:
                            reactants.append(coefficient + aliases['ModelSEED ID'])
                else:
                    reactants.append(element)

            products = []
            for element in products_list:
                element = element.strip()
                if re.search('(\d\s|\d\/\d\s)', element):
                    coefficient = stoich_coefficient.findall(element)[0]
                    if re.search('\/', coefficient):
                        numerator = re.findall('\d', coefficient)[0]
                        denominator = re.findall('\d', coefficient)[1]
                        numerical_coefficient = float(numerator) / float(denominator)
                        coefficient = '({}) '.format(numerical_coefficient)
                    else:
                        coefficient = '({}) '.format(re.findall('\d', coefficient)[0])
                else:
                    coefficient = ''
                    
                element = re.sub(stoich_coefficient, '', element)
                if element in described_compounds_list:
                    for compound, aliases in described_compounds.items():
                        if element.lower() == compound.lower() or element in aliases['alternative names']:
                            products.append(coefficient + aliases['ModelSEED ID'])
                else:
                    products.append(element)

            compounds = reactants + products
            compounds_list = reactants_list + products_list
        
        else:
            print('incorrect reaction: ', reaction)
            compounds = None
            
        # assemble the reaction string
        if len(reactants) > 1:
            reactants_string = ' + '.join(reactants)
        elif len(reactants) == 1:
            reactants_string = reactants[0]
        else:
            print('ERROR: The reaction __ {} __ has zero reactants'.format(reaction))

        if len(products) > 1:
            products_string = ' + '.join(products)
        elif len(products) == 1:
            products_string = products[0]
        else:
            print('\tERROR: The reaction __ {} __ has zero products'.format(reaction))
            print('\t{}'.format(products_list))

        reaction_string = reactants_string + ' <=> ' + products_string
            
        # format the reaction string according to ModelSEED conventions of the stoichiometric coefficients and the compartment signifier                        
        reaction_string = re.sub('((?<!\w)\d\s|\d\/\d\s)', r'({}) '.format(coefficient), reaction_string)
        reaction_string = re.sub('(?<=\d)(\s)(?=\))', '', reaction_string)
            
        if re.search('(?<=>|\+)(\s)(?=\w)', reaction_string):
            reaction_string = re.sub('(?<=>|\+)(\s)(?=\w)', ' (1) ', reaction_string)

        if not re.search('^( \d|\()', reaction_string):
            reaction_string = '(1) ' + reaction_string
            
        reaction_string = re.sub('(?<=\w)(\s)(?=<|\+)', '[0] ', reaction_string)
        reaction_string += '[0]'

        exchanged_reactions.add(reaction_string)
        if compounds is not None:
           
            # add the ModelSEED ID-exchange reaction string
            if all('cpd' in compound for compound in compounds):
                if reaction_string not in complete_reactions:
                    print(reaction_string)
                    complete_reactions.append(reaction_string)
                    
            # determine the set of described and undescribed compounds for each enzyme
            clean_compounds_list = []
            for compound in compounds_list:
                compound = compound.strip()
                compound = re.sub('(\(aq\)|\(l\)|\(sln\)|\(liq\))', '', compound)
                compound = re.sub('(\d\s|\d\/\d\s)', '', compound)
                if compound not in compounds_list:
                    clean_compounds_list.append(compound)
            
            described = all(compound in described_compounds_list for compound in clean_compounds_list)
            if described and reaction not in (undescribed_reactions and described_reactions):
                described_reactions.add(reaction)
            elif not described and reaction not in (undescribed_reactions and described_reactions):
                undescribed_reactions.add(reaction)
            else:
                repeated_reactions.add(reaction)

    # create unique reaction IDs for each compound
    reaction_ids = []
    for reaction in complete_reactions:
        if reaction not in reaction_id_pairs.values():
            reaction_id = 'TECRDB{}'.format(id_number)
            reaction_id_pairs[reaction_id] = reaction
            id_number += 1
        else:
            index = list(reaction_id_pairs.values()).index(reaction)
            reaction_id = list(reaction_id_pairs.keys())[index]
            print('--> repeated reaction, {}.'.format(reaction))
            
        reaction_ids.append(reaction_id)

    # add the enzyme-reactions associations to the respective dictionaries
    total_complete_reactions += complete_reactions    
    reactions[enzyme] = {'completely described reactions': {'reactions':complete_reactions, 'ids':reaction_ids},
                         'undescribed reactions': list(undescribed_reactions)}
    
# the parsed data is viewed
print('\n')
print('Total enzymes: {}'.format(len(described_enzyme_data.keys())))
print('Total reactions: {}'.format(len(exchanged_reactions)))                    
print('Total complete reactions: {}'.format(len(total_complete_reactions)))  
print('Total complete reactions set: {}'.format(len(set(total_complete_reactions))))
print('Previously complete reactions: {}'.format(len(set(list(tecrdb_reactions.keys())))))

print('\n')

completely_described = 0
undescribed = 0
for enzyme, data in reactions.items():
    if len(data['completely described reactions']) != 0 and len(data['undescribed reactions']) == 0:
        completely_described += 1
    else:
        undescribed += 1
        
print('enzymes with completely described reactions: ', completely_described)
print('enzymes with >0 undescribed reactions: ', undescribed)
print('repeated reactions for an enzyme: ', len(repeated_reactions))

# the parsed data is exported
with open('{}_TECR_reactions_in_ModelSEED.json'.format(date.today()), 'w') as output:
    json.dump(reactions, output, indent = 4)


# export the ModelSEED formatted reactions into a TSV file 
print('\n')
tsv_lines = []
for reaction_id in reaction_id_pairs:
    tsv_lines.append('\t'.join([reaction_id, reaction_id_pairs[reaction_id]]))
print('Total reactions in TSV: {}'.format(len(tsv_lines)))  
        
'''tsv_file = open('{}_TECR_to_ModelSEED_ID-rxn.tsv'.format(date.today()), 'w')
tsv_file.write('\t'.join(['ID', 'Equation']) + '\n' + '\n'.join(tsv_lines))
tsv_file.close()'''

identified alias:  l-aspartate
identified abbreviation:  fumarate
identified abbreviation:  ammonia
identified alias:  h2o
--> undescribed compound in ModelSEED, (s)-malate 
identified alias:  2-phospho-d-glycerate
identified alias:  phosphoenolpyruvate
identified abbreviation:  d-fructose 1,6-bisphosphate
identified alias:  glycerone phosphate
identified alias:  3-phospho-d-glycerate
identified alias:  phospho-l-arginine
identified alias:  l-arginine
identified alias:  orthophosphate
identified alias:  phosphocreatine
identified abbreviation:  creatine
--> undescribed compound in ModelSEED, (r)-3-phosphoglycerate 
--> undescribed compound in ModelSEED, (r)-glycerate 
identified alias:  pyruvate
identified abbreviation:  d-fructose 1-phosphate
identified alias:  d-fructose 6-phosphate
identified alias:  d-fructose 6-phosphate
identified abbreviation:  d-fructose
identified alias:  d-galactose 6-phosphate
identified abbreviation:  d-galactose
identified abbreviation:  d-glucose 6-phosph

identified alias:  d-glucosamine 6-phosphate
identified abbreviation:  d-glucosamine 1-phosphate
--> undescribed compound in ModelSEED, α-d-glucose 1,6-diphosphate 
--> undescribed compound in ModelSEED, 6-thioglucose 6-phosphate 
--> undescribed compound in ModelSEED, 6-thioglucose 1-phosphate 
identified alias:  sucrose
identified alias:  dadp
identified alias:  damp
identified alias:  datp
--> undescribed compound in ModelSEED, adp-n1-oxide 
--> undescribed compound in ModelSEED, atp-n1-oxide 
identified abbreviation:  adenosine 5'-tetraphosphate
identified alias:  citrate
identified alias:  cis-aconitate
identified alias:  isocitrate
identified alias:  isocitrate
identified abbreviation:  acetate
identified abbreviation:  acetyl phosphate
identified alias:  3-phospho-d-glyceroyl phosphate
identified alias:  (aminomethyl)phosphonate
--> undescribed compound in ModelSEED, (3-phospho-d-glycerol-aminomethyl)phosphonate 
--> undescribed compound in ModelSEED, (chloroethyl)phosphonate 
-

identified alias:  adpglucose
identified alias:  quinate
identified alias:  5-dehydroquinate
identified alias:  shikimate
identified abbreviation:  5-dehydroshikimate
identified alias:  d-glucono-1,5-lactone 6-phosphate
identified alias:  sedoheptulose 7-phosphate
identified abbreviation:  d-erythrose 4-phosphate
identified alias:  d-glyceraldehyde-3-phosphate
--> undescribed compound in ModelSEED, (r)-lactate 
identified abbreviation:  itp
identified abbreviation:  idp
--> undescribed compound in ModelSEED, atp4- 
identified abbreviation:  utp
identified alias:  ump
identified abbreviation:  orotidine 5'-phosphate
identified abbreviation:  orotate
--> undescribed compound in ModelSEED, 5-phospho-α-d-ribose 1-diphosphate 
--> undescribed compound in ModelSEED, thiopyrophosphate 
--> undescribed compound in ModelSEED, phosphoribosyl-1-o-(2-thiodiphosphate) 
--> undescribed compound in ModelSEED, (l-lysine)n 
--> undescribed compound in ModelSEED, (n-1) h2o 
identified alias:  l-lysine
-

--> undescribed compound in ModelSEED, d-erythrose 4-phospahte 
identified abbreviation:  2-hydroxy-3-oxopropanoate
identified alias:  dttp
--> undescribed compound in ModelSEED, dtdpglucose 
identified alias:  2-methyl-4-amino-5-hydroxymethylpyrimidine diphosphate
--> undescribed compound in ModelSEED, 4-methyl-5-(2-phosphonoxyethyl)-thiazole 
identified alias:  thiamine monophosphate
--> undescribed compound in ModelSEED, nω-phosphotaurocyamine 
--> undescribed compound in ModelSEED, l-tyrosylglycinamide 
identified alias:  l-tyrosine
identified alias:  benzoic acid
identified alias:  4-methyl-2-oxopentanoate
identified alias:  l-o-phosphoserine
identified abbreviation:  propanoyl-coa
identified alias:  methylmalonyl-coa
identified alias:  methylmalonyl-coa
identified abbreviation:  d-ribose 1-phosphate
identified alias:  6-phospho-2-dehydro-3-deoxy-d-gluconate
identified alias:  2-dehydro-3-deoxy-d-galactonate 6-phosphate
identified alias:  d-galactono-1,4-lactone
identified abbrevi

--> undescribed compound in ModelSEED, adenosine 3':5'-(cyclic)phosphate 
--> undescribed compound in ModelSEED, guanosine 3':5'-(cyclic)phosphate 
--> undescribed compound in ModelSEED, 2'-deoxyadenosine 3':5'-(cyclic)phosphate 
identified abbreviation:  2'-deoxyadenosine 5'-monophosphate
--> undescribed compound in ModelSEED, inosine 3':5'-(cyclic)phosphate 
--> undescribed compound in ModelSEED, uridine 3':5'-(cyclic)phosphate 
identified abbreviation:  \(\+\)-camphor
identified abbreviation:  \(\+\)-5-exo-hydroxycamphor
--> undescribed compound in ModelSEED, (r)-5-phosphomevalonate 
--> undescribed compound in ModelSEED, (r)-5-diphosphomevalonate 
--> undescribed compound in ModelSEED, atp-α-s 
--> undescribed compound in ModelSEED, adp-α-s 
--> undescribed compound in ModelSEED, atp-γ-s 
--> undescribed compound in ModelSEED, 2-thio-5-diphosphomevalonate 
identified abbreviation:  l,l-2,6-diaminoheptanedioate
identified alias:  meso-diaminoheptanedioate
--> undescribed compound in

--> undescribed compound in ModelSEED, nω-phospholombricine 
identified abbreviation:  disaccharide
identified abbreviation:  tetrasaccharide
--> undescribed compound in ModelSEED, trisaccharide 
identified alias:  d-arabitol
identified alias:  palatinose
identified alias:  d-turanose
identified alias:  chorismate
identified abbreviation:  isochorismate
identified abbreviation:  2-amino-2-deoxyisochorismate
identified alias:  retinyl palmitate
identified abbreviation:  retinol
identified alias:  palmitate
identified abbreviation:  4-amino-4-deoxychorismate
--> undescribed compound in ModelSEED, maltosyl-β-cyclomaltoheptaose 
identified alias:  diolein
identified alias:  oleic acid
--> undescribed compound in ModelSEED, monoolein 
identified alias:  triolein
--> undescribed compound in ModelSEED, n-decanoic acid glycerol diester 
--> undescribed compound in ModelSEED, n-decanoic acid 
--> undescribed compound in ModelSEED, n-decanoic acid glycerol monoester 
--> undescribed compound in 

(1) cpd00023[0] + (1) cpd00006[0] + (1) cpd00001[0] <=> (1) cpd00024[0] + (1) cpd00005[0] + (1) cpd00013[0]
(1) cpd00023[0] + (1) cpd00003[0] + (1) cpd00001[0] <=> (1) cpd00024[0] + (1) cpd00004[0] + (1) cpd00013[0]
--> repeated reaction, (1) cpd00023[0] + (1) cpd00003[0] + (1) cpd00001[0] <=> (1) cpd00024[0] + (1) cpd00004[0] + (1) cpd00013[0].

 arginosuccinate lyase

 dipeptidyl-peptidase I

 3(or 17)_-hydroxysteroid dehydrogenase

 ribose-5-phosphate isomerase
(1) cpd00101[0] <=> (1) cpd00171[0]

 nucleoside-diphosphate kinase
(1) cpd00002[0] + (1) cpd00090[0] <=> (1) cpd00008[0] + (1) cpd00068[0]
(1) cpd00002[0] + (1) cpd00031[0] <=> (1) cpd00008[0] + (1) cpd00038[0]

 ketotetraose-phosphate aldolase
(1) cpd02148[0] <=> (1) cpd00055[0] + (1) cpd00095[0]

 butyryl-CoA dehydrogenase
(1) cpd00120[0] + (1) cpd00120[0] + (1) cpd01206[0] <=> (1) cpd00650[0] + (1) cpd00650[0] + (1) cpd23609[0]

 glutamate-ammonia ligase

 dihydroorotase
(1) cpd00282[0] + (1) cpd00001[0] <=> (1) cpd00343[

 methylaspartate mutase
(1) cpd02273[0] <=> (1) cpd00023[0]

 oxamate carbamoyltransferase
(1) cpd00146[0] + (1) cpd01027[0] <=> (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd00009[0] + (1) cpd22507[0]

 methylmalonyl-CoA mutase

 tetrahydroxypteridine cycloisomerase
(1) cpd02033[0] <=> (1) cpd02112[0]

 thioredoxin reductase (NADPH)

 carnitine O-palmitoyltransferase
(1) cpd00134[0] + (1) cpd00266[0] <=> (1) cpd22528[0] + (1) cpd01915[0]

 mannose-1-phosphate guanylyltransferase

 D-lyxose ketol-isomerase
(1) cpd00368[0] <=> (1) cpd00259[0]
--> repeated reaction, (1) cpd00368[0] <=> (1) cpd00259[0].

 oxaloacetate tautomerase
(1) cpd00032[0] + (1) cpd00032[0] <=> (1) cpd02469[0]

 pancreatic ribonuclease

 deoxy)nucleoside-phosphate kinase

 papain

 rhamnulose-1-phosphate aldolase
(1) cpd00831[0] <=> (1) cpd00095[0] + (1) cpd00334[0]

 2-deoxy-D-gluconate dehydrogenase
(1) cpd01795[0] + (1) cpd00003[0] <=> (1) cpd02441[0] + (1) cpd00004[0]

 ureidoglycolate lyase

 N-acylglucosamine 2-e

(1) cpd01756[0] + (1) cpd00001[0] <=> (1) cpd00060[0] + (1) cpd29968[0]
(1) cpd33748[0] + (1) cpd00001[0] <=> (1) cpd29968[0] + (1) cpd00035[0]

 sorbitol-6-phosphate dehydrogenase (NADP+)

 pyrroline-5-carboxylate reductase

 GDP-D-mannose 3,5-epimerase

 serine-pyruvate transaminase
(1) cpd00035[0] + (1) cpd00145[0] <=> (1) cpd00054[0] + (1) cpd00020[0]

 glycerol kinase
(1) cpd00002[0] + (1) cpd00100[0] <=> (1) cpd00008[0] + (1) cpd00080[0]

 hypoxanthine phosphoribosyltransferase
(1) cpd00126[0] + (1) cpd00226[0] <=> (1) cpd00114[0] + (1) cpd00207[0]

 lactoylglutathione lyase

 malate dehydrogenase (decarboxylating)

 formate dehydrogenase (NADP+)
(1) cpd00047[0] + (1) cpd00006[0] + (1) cpd00001[0] <=> (1) cpd29976[0] + (1) cpd00005[0]

 dimethylmaleate hydratase

 thiamine pyridinylase

 methylenetetrahydrofolate dehydrogenase (NAD+)
(1) cpd00125[0] + (1) cpd00003[0] <=> (1) cpd00347[0] + (1) cpd00004[0]

 3-hydroxybutyryl-CoA dehyratase

 Ca2+-transporting ATPase
(1) cpd00002[0]

"tsv_file = open('{}_TECR_to_ModelSEED_ID-rxn.tsv'.format(date.today()), 'w')\ntsv_file.write('\t'.join(['ID', 'Equation']) + '\n' + '\n'.join(tsv_lines))\ntsv_file.close()"

# Brainstorming

Conceptualizing a scrapping code for KEGG, which would unambiguously identify the NIST compounds.

In [ ]:
import BeautifulSoup
import selenium
import json

compounds = json.load(open('2021-04-29_APF_complete thermodynamically described compounds for KEGG_01.json'))

num = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

general_delay = 2  #seconds



def click_element_type(n_type):
    global driver
    element = driver.find_element_by_xpath(n_type)
    element.click()
    time.sleep(general_delay)
    
    
def select_dropdown_id(n_id, n_choice):
    global driver
    element = Select(driver.find_element_by_id(n_id))
    element.select_by_visible_text(n_choice)
    time.sleep(general_delay)

    
def chunks(data, SIZE):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}
        
    
def scrape(compound):
    global driver
    global enzymes
    
    driver = webdriver.Chrome(executable_path=r".\chromedriver.exe")
    driver.get("https://www.genome.jp/kegg/")
    
    time.sleep(general_delay)
    
    # search field
    text_area = driver.find_element_by_name("text")
    text_area.send_keys(compound)
    time.sleep(general_delay)
    click_element_id("//input[@value='Search']")
    time.sleep(general_delay)
    
    # display all compound button
    click_element_type("/html/body/form/a[@href='javascript:MoreSearch(\"compound\")']")
    
    # acquire the list of results
    '''possible_compounds = []
    for div in click_element_type("/html/body/form/div[@style='width_600px']"):'''
    url = driver.current_url
    
    compounds[compound] = "no"
    driver.close()
    return
    
    time.sleep(general_delay)
    
    select_dropdown_id("max", "100")
    element = Select(driver.find_element_by_id("max"))
    element.select_by_visible_text("100")
    
    time.sleep(general_delay)

    else:
        compounds[compound] = "no"
        driver.close()
        return
    
    enzymes[enzyme_name] = "yes"
    
    driver.get("http://sabiork.h-its.org/newSearch/spreadsheetExport")
    
    time.sleep(15)
    
    click_element_id("excelExport")
    
    time.sleep(5)
    
    driver.close()
    

for compound in compounds:
    if compounds[compound] != "yes" and compounds[compound] != "no":
        flag = True
        while flag:
            try:
                scrape(compound)
                flag = False
            except:
                time.sleep(10)
            
        with open('compounds_to_collect_out.json', 'w') as outfile:
            json.dump(compounds, outfile, indent = 3)   
            outfile.close()

In [3]:
print('\xb0K')

°K


In [29]:
import re

compound = 'hi, yes, no'
re.sub('(^|, )(\S+)\w', '\u\1', ', {}, '.format(compound))
print(compound)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape (<ipython-input-29-f8fcd23c4c81>, line 4)